# Custom Training with YOLOv5

The notebook has the following:

* Download the drone dataset
* Train YOLOv5 to recognize the objects in our dataset
* Hyperparameter evolve
* Hyperparameter sweep


# Install Requirements

In [ ]:
# !git clone https://github.com/ultralytics/yolov5  # clone repo at v6.1
!git clone --depth 1 --branch v7.0 https://github.com/ultralytics/yolov5  # tag at v7.0

%cd yolov5
%pip install -qr requirements.txt # install dependencies
%cd ../

import torch
import os
from IPython.display import Image, clear_output

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")
!nvidia-smi

Cloning into 'yolov5'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 164 (delta 28), reused 143 (delta 24), pack-reused 0
Receiving objects: 100% (164/164), 967.39 KiB | 3.41 MiB/s, done.
Resolving deltas: 100% (28/28), done.
Note: switching to '915bbf294bb74c859f0b41f1c23bc395014ea679'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

/content/yolov5/yolov5
/content/yolov5
Setup complete. Using torch 2.1.0+cu118 (Tesla

# Download Dataset

In [ ]:
# Download drone dataset and unzip
# Dataset
# https://drive.google.com/file/d/1LnzXzxKhmk9YjsOaaKJKUu1KPBPaxcDs/view?usp=sharing

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt \
  --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V66KWmuK_j90h-ddxXPDmn94tmH2f_YW' -O- | \
  sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1V66KWmuK_j90h-ddxXPDmn94tmH2f_YW" -O uavgdrone.zip && rm -rf /tmp/cookies.txt
!unzip -qq uavgdrone.zip && rm uavgdrone.zip

--2023-10-31 10:28:34--  https://docs.google.com/uc?export=download&confirm=t&id=1V66KWmuK_j90h-ddxXPDmn94tmH2f_YW
Resolving docs.google.com (docs.google.com)... 74.125.24.101, 74.125.24.113, 74.125.24.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.24.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-cc-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/a5nv4b4hb5m1jqmfa5bjpn8ocmu7597o/1698748050000/17610671308944230943/*/1V66KWmuK_j90h-ddxXPDmn94tmH2f_YW?e=download&uuid=39359a85-d13c-4423-a20b-812781ffebfa [following]
--2023-10-31 10:28:35--  https://doc-08-cc-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/a5nv4b4hb5m1jqmfa5bjpn8ocmu7597o/1698748050000/17610671308944230943/*/1V66KWmuK_j90h-ddxXPDmn94tmH2f_YW?e=download&uuid=39359a85-d13c-4423-a20b-812781ffebfa
Resolving doc-08-cc-docs.googleusercontent.com (doc-08-cc-docs.googleusercontent.com)... 142.251.12.132, 2404

# Add logging

In [ ]:
%pip install -q wandb
import wandb
wandb.login()

wandb: Currently logged in as: xjp99v5 (ntu-uavg). Use `wandb login --relogin` to force relogin


True

# Configure yaml file

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

```
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# COCO128 dataset https://www.kaggle.com/ultralytics/coco128 (first 128 images from COCO train2017)
# Example usage: python train.py --data coco128.yaml
# parent
# ├── yolov5
# └── datasets
#     └── coco128  ← downloads here


# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../dataset  # dataset root dir
train: train/images  # train images (relative to 'path') 128 images
val: val/images  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 2  # number of classes
names: ['tello','DJI']  # class names


# Download script/URL (optional)
download: https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip
```

In [ ]:
# # Configuration for 2-class detection
# %cd yolov5
# yaml_content = """
# path: ../dataset
# train: train/images
# val: val/images
# test:  # test images (optional)
# nc: 2
# names: ['Tello','F450']
# """

# # Write the YAML content to a file
# with open('./data/drone.yaml', 'w') as file:
#     file.write(yaml_content)

In [ ]:
# Configuration for 3-class detection
%cd yolov5
yaml_content = """
path: ../uavgdrone/uavgdrone2c+1
train: train/images
val: val/images
test:  # test images (optional)
nc: 3
names: ['Tello','F450', 'QAV']
"""

# Write the YAML content to a file
with open('./data/drone.yaml', 'w') as file:
    file.write(yaml_content)

/content/yolov5/yolov5


# Train
Mount drive to save logs in Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!python train.py --img 640 --batch 40 --epochs 100 --data 'data/drone.yaml' --cache \
--weights yolov5m.pt \
--save-period 10 \
--project 'YOLOv5'
  # --resume wandb-artifact://prororo44/YOLOv5/29y433pz   # use this to resume training, change wandb artifact here

2023-10-31 10:29:47.744329: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 10:29:47.744384: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-31 10:29:47.744429: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-31 10:29:47.758757: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 10:29:50.539969: W tensorflow/c

In [ ]:
!python val.py --img 640 --batch 64 --data 'data/drone.yaml' --task speed\
--weights 'YOLOv5/exp/weights/best.pt' \
--project 'YOLOv5'

val: data=data/drone.yaml, weights=['YOLOv5/exp/weights/best.pt'], batch_size=64, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=speed, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=YOLOv5, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-0-g915bbf2 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20861016 parameters, 0 gradients, 47.9 GFLOPs
val: Scanning /content/yolov5/uavgdrone/uavgdrone2c+1/val/labels.cache... 356 images, 10 backgrounds, 0 corrupt: 100% 366/366 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 6/6 [00:19<00:00,  3.24s/it]
                   all        366        416      0.872       0.76      0.852      0.636
                 Tello        366        219      0.949      0.598       0.78      0.607
                  F450   

# Download `runs` contents to local

In [ ]:
!zip -r runs.zip runs
from google.colab import files
files.download('./runs.zip')

  adding: runs/ (stored 0%)
  adding: runs/train/ (stored 0%)
  adding: runs/train/exp/ (stored 0%)
  adding: runs/train/exp/train_batch2.jpg (deflated 5%)
  adding: runs/train/exp/R_curve.png (deflated 12%)
  adding: runs/train/exp/labels.jpg (deflated 29%)
  adding: runs/train/exp/opt.yaml (deflated 42%)
  adding: runs/train/exp/events.out.tfevents.1647096948.bb01e06de454.187.0 (deflated 30%)
  adding: runs/train/exp/results.csv (deflated 80%)
  adding: runs/train/exp/train_batch1.jpg (deflated 3%)
  adding: runs/train/exp/val_batch1_labels.jpg (deflated 7%)
  adding: runs/train/exp/PR_curve.png (deflated 13%)
  adding: runs/train/exp/labels_correlogram.jpg (deflated 26%)
  adding: runs/train/exp/P_curve.png (deflated 10%)
  adding: runs/train/exp/confusion_matrix.png (deflated 34%)
  adding: runs/train/exp/val_batch1_pred.jpg (deflated 6%)
  adding: runs/train/exp/val_batch0_pred.jpg (deflated 8%)
  adding: runs/train/exp/val_batch0_labels.jpg (deflated 9%)
  adding: runs/train/exp/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Hyperparams evolve

In [ ]:
!python train.py --img 640 --batch 64 --epochs 10 --data 'data/drone.yaml' --weights yolov5s.pt --cache --evolve 100 --name exp --resume

wandb: Currently logged in as: prororo44 (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=, data=data/drone.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=64, imgsz=600, rect=False, resume=True, nosave=False, noval=False, noautoanchor=False, evolve=20, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-22-g6dd82c0 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01171, lrf=0.01286, momentum=0.9024, weight_decay=0.00044, warmup_epochs=3.25428, warmup_momentum=0.92088, warmup_bias_lr=0.06849, box=0.05, cls=0.39876, cls_pw=0.8912, obj=1.03312, obj_

# Hyperparameter Sweep (wandb)
Edit yolov5/utils/loggers/wandb/sweep.yaml and change line 26 to

value: "data/drone.yaml"

In [ ]:
!wandb sweep utils/loggers/wandb/sweep.yaml

In [ ]:
!wandb agent prororo44/yolov5-utils_loggers_wandb/pcodc1f0 # change agent to the output of the previous cell

wandb: Starting wandb agent 🕵️
2022-03-06 07:04:26,299 - wandb.wandb_agent - INFO - Running runs: []
2022-03-06 07:05:22,132 - wandb.wandb_agent - INFO - Agent received command: run
2022-03-06 07:05:22,132 - wandb.wandb_agent - INFO - Agent starting run with config:
	anchor_t: 2.157335610242316
	batch_size: 64
	box: 0.06479086770242834
	cls: 2.626802282124721
	cls_pw: 0.8441024934507679
	copy_paste: 0.4761696411748192
	data: data/drone.yaml
	degrees: 34.125540365519356
	epochs: 10
	fl_gamma: 0.557421557719044
	fliplr: 0.34910074138424674
	flipud: 0.3423455163197734
	hsv_h: 0.042728871500794754
	hsv_s: 0.645737260170931
	hsv_v: 0.5093562439929117
	iou_t: 0.6529194512065811
	lr0: 0.037187397450131464
	lrf: 0.5158205749087567
	mixup: 0.6544744295392686
	momentum: 0.609511201123777
	mosaic: 0.22351529491922317
	obj: 3.979214406863176
	obj_pw: 1.1393689164458607
	perspective: 0.0009499227376058266
	scale: 0.5466765610926659
	shear: 1.3689936819540682
	translate: 0.01459674225451475
	warmup_

KeyboardInterrupt: ignored